In [28]:
import csv
import cv2
import numpy as np
import random
import os
import gc
import pandas as pd
import torch
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader
from torch.optim import lr_scheduler
from torch import nn
from torch.nn import functional as F

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
#read csv file
TRAIN_PATH = "/content/drive/MyDrive/Messi/Final/train.csv" #"/content/drive/MyDrive/Messi/Final/train.csv"
TEST_PATH = "/content/drive/MyDrive/Messi/Final/test.csv" #"/content/drive/MyDrive/Messi/Final/test.csv"
MODEL_PATH = "/content/drive/MyDrive/Messi/Final/model.pt" #"/content/drive/MyDrive/Messi/Final/model.pt"
train_data = pd.read_csv(TRAIN_PATH)
test_data = pd.read_csv(TEST_PATH)
cat_all = train_data.columns[2:25].tolist() #get out the feature name
val_ind = train_data.columns[25:26].tolist() #get out the target name

rep = {'material_5':5, 'material_6':6, 'material_7':7, 'material_8':8} #how to replace the material with number
#replace two columns in train_data and test_data
train_data = train_data.replace({'attribute_0': rep})
train_data = train_data.replace({'attribute_1': rep})
test_data = test_data.replace({'attribute_0': rep})
test_data = test_data.replace({'attribute_1': rep})

#fill in the nan value
#inspire by https://www.kaggle.com/code/tomjosephmo/logistic-regression-with-missing-values-accounted
for col in train_data.columns: #modify train data
    if not type(train_data[col][0]) == str: #if it is nan
        train_data[col].fillna(train_data[col].mean(), inplace=True)
        
for col in test_data.columns: #modify test data
    if not type(test_data[col][0]) == str: #if it is nan
        test_data[col].fillna(test_data[col].mean(), inplace=True)

device = "cuda" if torch.cuda.is_available() else "cpu" #if cuda is available use cuda
print(f"Using {device} device")

x_train = train_data[cat_all].values #get out the feature training data
y_train = train_data[val_ind].values #get out the target training data
y_train = y_train.flatten()
x_test = test_data[cat_all].values #get out the feature test data

Using cpu device


# 新增區段

# 新增區段

In [31]:
class TPS_train_dataset(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = torch.tensor(x_data, dtype=torch.float32)
        self.y_data = torch.tensor(y_data, dtype=torch.float32)
        self.length = self.x_data.shape[0]

    def __getitem__(self, ind):
        return self.x_data[ind], self.y_data[ind] #return x data and y data

    def __len__(self):
        return self.length
class TPS_test_dataset(Dataset):
    def __init__(self, x_data):
        self.x_data = torch.tensor(x_data, dtype=torch.float32)
        self.length = self.x_data.shape[0]

    def __getitem__(self, ind):
        return self.x_data[ind] #test data dont have target feature so only return x

    def __len__(self):
        return self.length


In [32]:
#three fully connected layer
class Model(nn.Module):
    def __init__(self, num_feature):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(num_feature, 32)
        self.fc2 = nn.Linear(32, 64)
        self.fc3 = nn.Linear(64, 1) #make output of the network to right dimension

    def forward(self, x_data):
        x_data = torch.relu(self.fc1(x_data)) #put in relu activation function
        x_data = torch.relu(self.fc2(x_data))
        x_data = torch.sigmoid(self.fc3(x_data)) #use sigmoid to make value between 0-1
        return x_data #return model

In [33]:
#load test data
test_ds = TPS_test_dataset(x_test)
test_dl = DataLoader(test_ds, batch_size = 1, shuffle = False)
        

In [34]:
ind = np.array([i for i in range(26570, 47345)])
result = []
model = Model(num_feature = 23)
model.load_state_dict(torch.load(MODEL_PATH))
model.eval()
for x_data in test_dl:
    pred = model(x_data) #predict the result
    pred = pred.detach().numpy()
    pred = pred.flatten()
    result = np.concatenate((result, pred)) #concate all result together

df = pd.DataFrame({"id":ind, "failure":result}, columns = ["id", "failure"]) #set dataframe
df.to_csv(f"109550147_submission.csv", index=False) #index set to false so that csv file dont have one extra column
df

,id,failure
0,26570,0.219425
1,26571,0.157592
2,26572,0.185598
3,26573,0.204562
4,26574,0.310453
...,...,...
20770,47340,0.251734
20771,47341,0.136056
20772,47342,0.121570
20773,47343,0.239759
